### Define function to transform GeoJSON data to pixel data

In [13]:
from osgeo import gdal, osr, ogr
import numpy as np
import os
import csv
import subprocess
import math
import geopandas as gpd
import shapely
from shapely.geometry import Point
from pyproj import Proj, transform
from fiona.crs import from_epsg
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.linestring import LineString
from shapely.geometry.multilinestring import MultiLineString


def latlon2pixel(lat, lon, input_raster='', targetsr='', geom_transform=''):
    # type: (object, object, object, object, object) -> object

    sourcesr = osr.SpatialReference()
    sourcesr.ImportFromEPSG(4326)

    geom = ogr.Geometry(ogr.wkbPoint)
    geom.AddPoint(lon, lat)

    if targetsr == '':
        src_raster = gdal.Open(input_raster)
        targetsr = osr.SpatialReference()
        targetsr.ImportFromWkt(src_raster.GetProjectionRef())
    coord_trans = osr.CoordinateTransformation(sourcesr, targetsr)
    if geom_transform == '':
        src_raster = gdal.Open(input_raster)
        transform = src_raster.GetGeoTransform()
    else:
        transform = geom_transform

    x_origin = transform[0]
    # print(x_origin)
    y_origin = transform[3]
    # print(y_origin)
    pixel_width = transform[1]
    # print(pixel_width)
    pixel_height = transform[5]
    # print(pixel_height)
    geom.Transform(coord_trans)
    # print(geom.GetPoint())
    x_pix = (geom.GetPoint()[0] - x_origin) / pixel_width
    y_pix = (geom.GetPoint()[1] - y_origin) / pixel_height

    return (x_pix, y_pix)

### Transform GeoJSON to Pixel Array

In [14]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Reference: 
# https://medium.com/the-downlinq/getting-started-with-spacenet-data-827fd2ec9f53
# https://gist.github.com/avanetten/534e6f341b0670be2ea74ce39d68226a#file-geojson_to_pixel_arr-py

from osgeo import gdal, osr
import numpy as np
import json
import sys
import os

# Import geo-tools from SpaceNetChallenge/utilities.
# https://github.com/motokimura/spacenet_utilities
# __this_dir = os.path.dirname(__file__)
# sys.path.append(os.path.normpath(os.path.join(__this_dir, "utilities/python")))


def geojson_to_pixel_arr(raster_file, geojson_file, pixel_ints=True, verbose=False):
	'''
	Tranform geojson file into array of points in pixel (and latlon) coords
	pixel_ints = 1 sets pixel coords as integers
	'''
	
	# load geojson file
	with open(geojson_file) as f:
		geojson_data = json.load(f)

	# load raster file and get geo transforms
	src_raster = gdal.Open(raster_file)
	targetsr = osr.SpatialReference()
	targetsr.ImportFromWkt(src_raster.GetProjectionRef())
		
	geom_transform = src_raster.GetGeoTransform()
	
	# get latlon coords
	latlons = []
	types = []
	for feature in geojson_data['features']:
		coords_tmp = feature['geometry']['coordinates'][0]
		type_tmp = feature['geometry']['type']
		if verbose: 
			print("features:", feature.keys())
			print("geometry:features:", feature['geometry'].keys())

			#print("feature['geometry']['coordinates'][0]", z)
		latlons.append(coords_tmp)
		types.append(type_tmp)
		#print(feature['geometry']['type'])
	
	# convert latlons to pixel coords
	pixel_coords = []
	latlon_coords = []
	for i, (poly_type, poly0) in enumerate(zip(types, latlons)):
		
		if poly_type.upper() == 'MULTIPOLYGON':
			#print("oops, multipolygon")
			for poly in poly0:
				poly=np.array(poly)
				if verbose:
					print("poly.shape:", poly.shape)
					
				# account for nested arrays
				if len(poly.shape) == 3 and poly.shape[0] == 1:
					poly = poly[0]
					
				poly_list_pix = []
				poly_list_latlon = []
				if verbose: 
					print("poly", poly)
				for coord in poly:
					if verbose: 
						print("coord:", coord)
					lon, lat, z = coord 
					px, py = latlon2pixel(lat, lon, input_raster=src_raster,
										 targetsr=targetsr, 
										 geom_transform=geom_transform)
					poly_list_pix.append([px, py])
					if verbose:
						print("px, py", px, py)
					poly_list_latlon.append([lat, lon])
				
				if pixel_ints:
					ptmp = np.rint(poly_list_pix).astype(int)
				else:
					ptmp = poly_list_pix
				pixel_coords.append(ptmp)
				latlon_coords.append(poly_list_latlon)            

		elif poly_type.upper() == 'POLYGON':
			poly=np.array(poly0)
			if verbose:
				print("poly.shape:", poly.shape)
				
			# account for nested arrays
			if len(poly.shape) == 3 and poly.shape[0] == 1:
				poly = poly[0]
				
			poly_list_pix = []
			poly_list_latlon = []
			if verbose: 
				print("poly", poly)
			for coord in poly:
				if verbose: 
					print("coord:", coord)
				lon, lat, z = coord 
				px, py = latlon2pixel(lat, lon, input_raster=src_raster,
									 targetsr=targetsr, 
									 geom_transform=geom_transform)
				poly_list_pix.append([px, py])
				if verbose:
					print("px, py", px, py)
				poly_list_latlon.append([lat, lon])
			
			if pixel_ints:
				ptmp = np.rint(poly_list_pix).astype(int)
			else:
				ptmp = poly_list_pix
			pixel_coords.append(ptmp)
			latlon_coords.append(poly_list_latlon)
			
		else:
			print("Unknown shape type in coords_arr_from_geojson()")
			return
			
	return pixel_coords, latlon_coords

### Visualize data

In [15]:

#!/usr/bin/env python2
# -*- coding: utf-8 -*-

from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
import numpy as np

###############################################################################
def plot_truth_coords(input_image, pixel_coords,   
                  figsize=(8,8), plot_name='',
                  add_title=False, poly_face_color='orange', 
                  poly_edge_color='red', poly_nofill_color='blue', cmap='bwr'):
    '''Plot ground truth coordinaates, pixel_coords should be a numpy array'''
    
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*figsize[0], figsize[1]))
    
    if add_title:
        suptitle = fig.suptitle(plot_name.split('/')[-1], fontsize='large')
    
    # create patches
    patches = []
    patches_nofill = []
    if len(pixel_coords) > 0:
        # get patches    
        for coord in pixel_coords:
            patches_nofill.append(Polygon(coord, facecolor=poly_nofill_color, 
                                          edgecolor=poly_edge_color, lw=3))
            patches.append(Polygon(coord, edgecolor=poly_edge_color, fill=True, 
                                   facecolor=poly_face_color))
        p0 = PatchCollection(patches, alpha=0.25, match_original=True)
        #p1 = PatchCollection(patches, alpha=0.75, match_original=True)
        p2 = PatchCollection(patches_nofill, alpha=0.75, match_original=True)
                   
    # ax0: raw image
    ax0.imshow(input_image)
    if len(patches) > 0:
        ax0.add_collection(p0)
    ax0.set_title('Input Image + Ground Truth Buildings') 
    
    # truth polygons
    zero_arr = np.zeros(input_image.shape[:2])
    # set background to white?
    #zero_arr[zero_arr == 0.0] = np.nan
    ax1.imshow(zero_arr, cmap=cmap)
    if len(patches) > 0:
        ax1.add_collection(p2)
    ax1.set_title('Ground Truth Building Polygons')
        
    #plt.axis('off')
    plt.tight_layout()
    if add_title:
        suptitle.set_y(0.95)
        fig.subplots_adjust(top=0.96)
    #plt.show()
 
    if len(plot_name) > 0:
        plt.savefig(plot_name)
    
    return patches, patches_nofill
    
